In [82]:
import pandas as pd

In [49]:
# Cargar el archivo CSV
file_path = 'https://raw.githubusercontent.com/Adalab/da-prueba-tecnica-transporte/main/data/Transporte_Publico_Espana.csv'
transporte_es = pd.read_csv(file_path)

In [50]:
# Examinamos las primeras filas
transporte_es.head()

,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
0,1,2023-01-01 00:00:00,Barcelona - Bilbao,NaN,not_available,999,Tranvía,Sevilla,Sunday
1,2,2023-01-01 00:01:00,Malaga - Sevilla,99.0,192.0,58,Tren,Barcelona,Sunday
2,3,2023-01-01 00:02:00,Madrid - Zaragoza,10.0,94.0,119,Autobús,Madrid,Sunday
3,4,2023-01-01 00:03:00,Sevilla - Madrid,96.0,650.0,2,Metro,Malaga,Sunday
4,5,2023-01-01 00:04:00,Madrid - Malaga,22.0,335.0,51,Autobús,Sevilla,Sunday


In [51]:
# Analizamos estadísticas básicas
transporte_es.describe(include='all')

,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
count,100100.000000,100100,100100,99099.000000,99767,100100.000000,100100,100100,100100
unique,NaN,100000,10,NaN,706,NaN,4,7,7
top,NaN,2023-01-01 00:00:00,Bilbao - Madrid,NaN,not_available,NaN,Tranvía,Madrid,Sunday
freq,NaN,2,10201,NaN,335,NaN,25084,14423,14500
mean,49950.599900,NaN,NaN,54.465171,NaN,64.044685,NaN,NaN,NaN
std,28896.352622,NaN,NaN,25.956460,NaN,74.795155,NaN,NaN,NaN
min,1.000000,NaN,NaN,10.000000,NaN,0.000000,NaN,NaN,NaN
25%,24925.750000,NaN,NaN,32.000000,NaN,29.000000,NaN,NaN,NaN
50%,49950.500000,NaN,NaN,54.000000,NaN,59.000000,NaN,NaN,NaN
75%,74975.250000,NaN,NaN,77.000000,NaN,90.000000,NaN,NaN,NaN


Analizando el conteo total y los IDs, único, deducimos que hay datos duplicados, por lo que procedemos a su comprobación y eliminación.

In [52]:
# Al ver el conteo 100100 y el máximo id 100K, deducimos que hay duplicados, por lo que procedemos a su detección y los eliminamos
# Detectar duplicados
num_duplicates = transporte_es.duplicated().sum()
print(num_duplicates)
# Eliminar duplicados
transporte_es = transporte_es.drop_duplicates()
transporte_es.describe(include='all')


100


,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
count,100000.000000,100000,100000,99000.000000,99667,100000.00000,100000,100000,100000
unique,NaN,100000,10,NaN,706,NaN,4,7,7
top,NaN,2023-01-01 00:00:00,Bilbao - Madrid,NaN,not_available,NaN,Tranvía,Madrid,Sunday
freq,NaN,1,10191,NaN,334,NaN,25059,14405,14400
mean,50000.500000,NaN,NaN,54.465293,NaN,64.04027,NaN,NaN,NaN
std,28867.657797,NaN,NaN,25.954694,NaN,74.76642,NaN,NaN,NaN
min,1.000000,NaN,NaN,10.000000,NaN,0.00000,NaN,NaN,NaN
25%,25000.750000,NaN,NaN,32.000000,NaN,29.00000,NaN,NaN,NaN
50%,50000.500000,NaN,NaN,54.000000,NaN,59.00000,NaN,NaN,NaN
75%,75000.250000,NaN,NaN,77.000000,NaN,90.00000,NaN,NaN,NaN


## Análisis y corrección de tipos

In [53]:
print(transporte_es.dtypes)

ID_Viaje                    int64
Fecha                      object
Ruta                       object
Numero_Pasajeros          float64
Duracion_Viaje_Minutos     object
Retraso_Minutos             int64
Tipo_Transporte            object
Region                     object
Dia_Semana                 object
dtype: object


### Fecha
Tenemos la fecha en formato texto, tenemos que tiparla apropiadamente.

In [54]:
transporte_es['Fecha'] = pd.to_datetime(transporte_es['Fecha'], format='%Y-%m-%d %H:%M:%S')
print(transporte_es.dtypes)
transporte_es.tail(5)

ID_Viaje                           int64
Fecha                     datetime64[ns]
Ruta                              object
Numero_Pasajeros                 float64
Duracion_Viaje_Minutos            object
Retraso_Minutos                    int64
Tipo_Transporte                   object
Region                            object
Dia_Semana                        object
dtype: object


,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
99995,99996,2023-03-11 10:35:00,Malaga - Sevilla,89.0,566.0,38,Autobús,Valencia,Saturday
99996,99997,2023-03-11 10:36:00,Madrid - Zaragoza,26.0,621.0,106,Metro,Zaragoza,Saturday
99997,99998,2023-03-11 10:37:00,Valencia - Sevilla,95.0,174.0,54,Tren,Valencia,Saturday
99998,99999,2023-03-11 10:38:00,Madrid - Zaragoza,83.0,248.0,96,Tranvía,Madrid,Saturday
99999,100000,2023-03-11 10:39:00,Malaga - Sevilla,72.0,478.0,94,Metro,Sevilla,Saturday


In [60]:
# Obtenemos rango de fechas
min_fecha = transporte_es['Fecha'].min()
max_fecha = transporte_es['Fecha'].max()

# Comprobamos que hay un valor para cada minuto
difference_minutes = round((max_fecha - min_fecha).total_seconds() / 60) + 1
print('Fecha mínima:', min_fecha)
print('Fecha máxima:', max_fecha)
print('La diferencia entre la fecha-hora máxima y mínima es', difference_minutes)

Fecha mínima: 2023-01-01 00:00:00
Fecha máxima: 2023-03-11 10:39:00
La diferencia entre la fecha-hora máxima y mínima es 100000


### Duracion_Viaje_Minutos
Debemos convertirlo a entero

In [61]:
transporte_es['Duracion_Viaje_Minutos'] = pd.to_numeric(transporte_es['Duracion_Viaje_Minutos'], errors='coerce')  
print(transporte_es.dtypes)

ID_Viaje                           int64
Fecha                     datetime64[ns]
Ruta                              object
Numero_Pasajeros                 float64
Duracion_Viaje_Minutos           float64
Retraso_Minutos                    int64
Tipo_Transporte                   object
Region                            object
Dia_Semana                        object
Origen                            object
Destino                           object
dtype: object


### Examinamos las rutas

Separamos origen y destino

In [62]:
# Nº de rutas distintas
countdistinct_ruta_values = transporte_es['Ruta'].nunique()
countdistinct_ruta_values

10

In [57]:
distinct_ruta_values = transporte_es['Ruta'].unique()
print(distinct_ruta_values)

['Barcelona - Bilbao' 'Malaga - Sevilla' 'Madrid - Zaragoza'
 'Sevilla - Madrid' 'Madrid - Malaga' 'Madrid - Barcelona'
 'Bilbao - Madrid' 'Valencia - Sevilla' 'Barcelona - Valencia'
 'Zaragoza - Barcelona']


In [58]:
# Extraer de 'Ruta' las columnas nuevas: 'Origen' y 'Destino'
transporte_es[['Origen', 'Destino']] = transporte_es['Ruta'].str.split(' - ', expand=True)


In [65]:
# Extraer datos temporales a partir de la fecha
transporte_es['Minuto'] = transporte_es['Fecha'].dt.minute
transporte_es['Hora'] = transporte_es['Fecha'].dt.hour
transporte_es['Dia'] = transporte_es['Fecha'].dt.day
transporte_es['Mes'] = transporte_es['Fecha'].dt.month


In [66]:
# Creamos una nueva columna fecha sin hora para reducir la granularidad de la distribución
transporte_es['Fecha_Sin_Hora'] = transporte_es['Fecha'].dt.date



### Análisis de valores faltantes
Se examinan los nulos con un paquete de R: skimr, que permite automatizar la tarea.
Se prepara también el análsis para introducir código R

In [67]:
import importlib.util
package_name = 'rpy2'

if importlib.util.find_spec(package_name) is None:
    !pip install {package_name}

%load_ext rpy2.ipython



The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [72]:
%%R
# a partir de la línea anterior, el chunk reconoce código R
# se instala el paquete skimr en el entorno si no existiera y se carga
if (!requireNamespace("skimr", quietly = TRUE)) {
    install.packages("skimr")
} 
# Cargar el paquete 'skimr'
library(skimr)

In [73]:
# Movemos el dataframe a R
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [74]:
ro.globalenv['transporte_es'] = transporte_es

In [75]:
%%R
library(skimr)
skim(transporte_es) # análisis en modo texto de la distribución, incluyendo tasa de valores perdidos

── Data Summary ────────────────────────
                           Values       
Name                       transporte_es
Number of rows             100000       
Number of columns          16           
_______________________                 
Column type frequency:                  
  character                6            
  numeric                  9            
  POSIXct                  1            
________________________                
Group variables            None         

── Variable type: character ────────────────────────────────────────────────────
  skim_variable   n_missing complete_rate min max empty n_unique whitespace
1 Ruta                    0             1  15  20     0       10          0
2 Tipo_Transporte         0             1   4   7     0        4          0
3 Region                  0             1   6   9     0        7          0
4 Dia_Semana              0             1   6   9     0        7          0
5 Origen                  0             1   6 

Se detecta un 1 % de valores perdidos en Numero_Pasajeros y un 0.67 % en Duración_Viaje_Minutos.
Además, si se da un caso, es más frecuente que se de el otro. De hecho, la mitad de los nulos de duración, se dan cuando se pierde el nº de pasajeros.

In [79]:
%%R
library(dplyr)

# Filtrando por situaciones con nº pasajeros faltante:
transporte_es |>
  dplyr::filter(is.na(Numero_Pasajeros)) |>
    skim()

── Data Summary ────────────────────────
                           Values                      
Name                       dplyr::filter(transporte_...
Number of rows             1000                        
Number of columns          16                          
_______________________                                
Column type frequency:                                 
  character                6                           
  numeric                  9                           
  POSIXct                  1                           
________________________                               
Group variables            None                        

── Variable type: character ────────────────────────────────────────────────────
  skim_variable   n_missing complete_rate min max empty n_unique whitespace
1 Ruta                    0             1  15  20     0       10          0
2 Tipo_Transporte         0             1   4   7     0        4          0
3 Region                  0      

Además, la media de los retrasos se eleva considerablemente. Pasando de 64 min a 530

### En lugar de imputar nulos o eliminarlos, se van a analizar:

In [81]:
transporte_es['duracion_error'] = transporte_es['Duracion_Viaje_Minutos'].apply(lambda x: 'ok' if pd.notnull(x) else 'error')

In [46]:
transporte_es.head()

,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana,Origen,Destino,Minuto,Hora,Dia,Mes,Fecha_Sin_Hora,npasajeros_error,duracion_error
0,1,2023-01-01 00:00:00,Barcelona - Bilbao,NaN,NaN,999,Tranvía,Sevilla,Sunday,Barcelona,Bilbao,0,0,1,1,2023-01-01,error,error
1,2,2023-01-01 00:01:00,Malaga - Sevilla,99.0,192.0,58,Tren,Barcelona,Sunday,Malaga,Sevilla,1,0,1,1,2023-01-01,ok,ok
2,3,2023-01-01 00:02:00,Madrid - Zaragoza,10.0,94.0,119,Autobús,Madrid,Sunday,Madrid,Zaragoza,2,0,1,1,2023-01-01,ok,ok
3,4,2023-01-01 00:03:00,Sevilla - Madrid,96.0,650.0,2,Metro,Malaga,Sunday,Sevilla,Madrid,3,0,1,1,2023-01-01,ok,ok
4,5,2023-01-01 00:04:00,Madrid - Malaga,22.0,335.0,51,Autobús,Sevilla,Sunday,Madrid,Malaga,4,0,1,1,2023-01-01,ok,ok


In [47]:
# Se guarda el csv para analizarlo más detenidamente con Power BI
cleaned_dataset_path = "cleaned_transporte_es.csv"
transporte_es.to_csv(cleaned_dataset_path, index=False)



In [ ]:
# Se envía el dataset limpio a un servidor Postgresql, cuyos datos de conexión se encuentran en variables de entorno del sistema
import os
from sqlalchemy import create_engine

# Obtener credenciales de las variables de entorno
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Crear cadena de conexión
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Crear SQLAlchemy engine
engine = create_engine(connection_string)

# Tabla
table_name = 'transporte_es'

# Enviar el data frame a la bbdd
transporte_es.to_sql(table_name, engine, if_exists='replace', index=False)

Conclusión:
El dataset parece sintético, la granularidad es a minuto por viaje. Lo interesante a analizar es cuándo se producen esos errores en el conteo del nº de pasajeros y en la duración.
Cuando hay errores en el conteo de pasajeros, los retrasos aumentan significativamente.
En la ruta Barcelona-Valencia es la más problemática en cuanto a duración de los retrasos
La ruta Málaga-Sevilla, es la menor en cuanto a la duración.
Cuando no hay errores en conteo, los retrasos siguen siendo muy altos en promedio, del orden de 59 minutos
Analizando series temporales, se observa que los errores de conteo se producen de forma cíclica